In [1]:
# # prompt: download multinli dataset

# from google.colab import drive
# drive.mount('/content/drive')

# %cd drive/My Drive/TULANE/SPRING2024/CMPS4730/Project-NLI/
# # Peter run a seperate cell with your path

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
!cd /Users/petersapountzis/Desktop/tulane/spring2024/cmps4730/project-nli

In [3]:
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7")
model = AutoModelForSequenceClassification.from_pretrained("MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7")
print("Tokenizer and model loaded successfully")
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


Tokenizer and model loaded successfully


In [4]:
mnli_train = pd.read_csv('/Users/petersapountzis/Desktop/tulane/spring2024/cmps4730/project-nli/data/mnli_data/train.csv')
mnli_valid = pd.read_csv('/Users/petersapountzis/Desktop/tulane/spring2024/cmps4730/project-nli/data/mnli_data/validation_matched.csv')
mnli_train[:5]

,promptID,pairID,premise,premise_binary_parse,premise_parse,hypothesis,hypothesis_binary_parse,hypothesis_parse,genre,label
0,31193,31193n,Conceptually cream skimming has two basic dime...,( ( Conceptually ( cream skimming ) ) ( ( has ...,(ROOT (S (NP (JJ Conceptually) (NN cream) (NN ...,Product and geography are what make cream skim...,( ( ( Product and ) geography ) ( ( are ( what...,(ROOT (S (NP (NN Product) (CC and) (NN geograp...,government,1
1,101457,101457e,you know during the season and i guess at at y...,( you ( ( know ( during ( ( ( the season ) and...,(ROOT (S (NP (PRP you)) (VP (VBP know) (PP (IN...,You lose the things to the following level if ...,( You ( ( ( ( lose ( the things ) ) ( to ( the...,(ROOT (S (NP (PRP You)) (VP (VBP lose) (NP (DT...,telephone,0
2,134793,134793e,One of our number will carry out your instruct...,( ( One ( of ( our number ) ) ) ( ( will ( ( (...,(ROOT (S (NP (NP (CD One)) (PP (IN of) (NP (PR...,A member of my team will execute your orders w...,( ( ( A member ) ( of ( my team ) ) ) ( ( will...,(ROOT (S (NP (NP (DT A) (NN member)) (PP (IN o...,fiction,0
3,37397,37397e,How do you know? All this is their information...,( ( How ( ( ( do you ) know ) ? ) ) ( ( All th...,(ROOT (S (SBARQ (WHADVP (WRB How)) (SQ (VBP do...,This information belongs to them.,( ( This information ) ( ( belongs ( to them )...,(ROOT (S (NP (DT This) (NN information)) (VP (...,fiction,0
4,50563,50563n,yeah i tell you what though if you go price so...,( yeah ( i ( ( tell you ) ( what ( ( though ( ...,(ROOT (S (VP (VB yeah) (S (NP (FW i)) (VP (VB ...,The tennis shoes have a range of prices.,( ( The ( tennis shoes ) ) ( ( have ( ( a rang...,(ROOT (S (NP (DT The) (NN tennis) (NNS shoes))...,telephone,1


In [5]:
premise = "Im studying in the library"
hypothesis = "I'm on campus"

input = tokenizer(premise, hypothesis, truncation=True, return_tensors="pt")
output = model(input["input_ids"].to(device))  # device = "cuda:0" or "cpu"
prediction = torch.softmax(output["logits"][0], -1).tolist()
label_names = ["entailment", "neutral", "contradiction"]
prediction = {name: round(float(pred) * 100, 1) for pred, name in zip(prediction, label_names)}
print(prediction)

{'entailment': 38.0, 'neutral': 6.0, 'contradiction': 56.1}


In [6]:
import os

# Disable tokenizers parallelism to avoid the warning
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [7]:
import psutil

# Get the virtual memory structure using psutil.virtual_memory()
vmem = psutil.virtual_memory()

# Total memory
print(f'Total memory: {vmem.total / (1024**3):.2f} GB')

# Available memory
print(f'Available memory: {vmem.available / (1024**3):.2f} GB')

# Used memory
print(f'Used memory: {vmem.used / (1024**3):.2f} GB')

# Memory percent used
print(f'Memory percent used: {vmem.percent}%')

Total memory: 18.00 GB
Available memory: 1.86 GB
Used memory: 2.23 GB
Memory percent used: 89.7%


In [11]:
mnli_train['premise'] = mnli_train['premise'].astype(str)
mnli_train['hypothesis'] = mnli_train['hypothesis'].astype(str)

mnli_train = mnli_train[:10]

tokenized_input = tokenizer(mnli_train['premise'].tolist(), mnli_train['hypothesis'].tolist(), padding=True, truncation=True, return_tensors="pt")


In [12]:
from torch.utils.data import Dataset, DataLoader
import torch

class MNLIDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Assuming mnli_train['label'] contains the integer labels for your dataset
labels = mnli_train['label'].values

dataset = MNLIDataset(tokenized_input, labels)


In [13]:
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)



In [15]:
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = CrossEntropyLoss()

In [18]:
from tqdm.auto import tqdm

epochs = 1  # Number of training epochs

for epoch in range(3):
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_loader, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:
        # Move batch to device
        batch = {k: v.to(device) for k, v in batch.items()}
        
        # Forward pass
        outputs = model(**batch)
        loss = outputs.loss if outputs.loss is not None else loss_fn(outputs.logits, batch['labels'])
        
        # Backward pass
        loss.backward()
        
        # Update parameters and zero gradients
        optimizer.step()
        optimizer.zero_grad()
        
        total_loss += loss.item()
        progress_bar.set_postfix({'loss': '{:.3f}'.format(loss.item())})
    
    # Calculate average loss over the epoch
    avg_loss = total_loss / len(train_loader)
    print('Epoch: {} | Avg. Loss: {:.3f}'.format(epoch+1, avg_loss))

Epoch 0:   0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 1 | Avg. Loss: 0.010


Epoch 1:   0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 2 | Avg. Loss: 0.008


Epoch 2:   0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 3 | Avg. Loss: 0.004


In [23]:
# Assuming `tokenizer` has already been defined and loaded as shown earlier
mnli_valid = mnli_valid[:10]

tokenized_valid = tokenizer(
    mnli_valid['premise'].tolist(), 
    mnli_valid['hypothesis'].tolist(), 
    padding=True, 
    truncation=True, 
    return_tensors="pt"
)


In [24]:
valid_labels = torch.tensor(mnli_valid['label'].values)


In [26]:
valid_dataset = MNLIDataset(tokenized_valid, valid_labels)
val_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)  # Adjust batch_size as necessary


In [27]:
# Assuming you have a validation DataLoader named `val_loader`
model.eval()
total_eval_accuracy = 0
for batch in val_loader:
    batch = {k: v.to(device) for k, v in batch.items()}
    
    with torch.no_grad():
        outputs = model(**batch)
    
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    accuracy = (predictions == batch['labels']).cpu().numpy().mean() * 100
    total_eval_accuracy += accuracy

avg_val_accuracy = total_eval_accuracy / len(val_loader)
print('Validation Accuracy: {:.2f}%'.format(avg_val_accuracy))


/var/folders/nx/h6fkpnxn1qz95969ny71ltf00000gn/T/ipykernel_6145/3207049264.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


Validation Accuracy: 80.00%
